# 函数式编程

函数式编程最大的特点是将“函数”作为一等公民，同时保证所有变量都是不可变的immutable，这样就保证了程序的稳定健壮，可测试性好。但同时，也因为这些特性，完全的函数式编程比较难。

在Python中，也对函数式编程有所支持，最主要的就是匿名函数lambda，map、filter和reduce这三个高阶函数的支持。

匿名函数适用在简单场景，比如配合map、reduce完成某个任务，或者通过列表组合方式创建新的列表等。

In [5]:
from functools import reduce

l = [1, 2, 3, 4, 5]
double_l = map(lambda x: x * 2, l) # [2， 4， 6， 8， 10]
odd_l = filter(lambda x: x % 2 == 0, l) # [2, 4]
acc_l = reduce(lambda x, y: x * y, l)

print(f"raw list is {l}\n"
     f"odd list is {list(odd_l)}\n"
     f"accumutive list is {acc_l}")

raw list is [1, 2, 3, 4, 5]
odd list is [2, 4]
accumutive list is 120


# 1.装饰器

如其名字所言，“装饰器”就是为了“装饰”某个函数或者类，装饰的目的是为了增强函数、类的功能。在Python中，装饰器使用的非常广泛，比如用作日志记录、缓存等。

要想装饰器起作用，至少需要满足一下条件：
1. 函数为一等公民，可以被作为参数传递，也可以被作为返回值返回
2. 支持嵌套函数，即可以在函数内定义函数

另外，Python中的装饰器也比较多样，可以是对原函数简单的包装，也可以是多个嵌套在一起，还有类装饰器。

## 1.1 简单的装饰器

In [1]:
# 装饰器的“原始写法”

def my_decorator(func):
    def wrapper():
        print('my decorator works...')
        func()
    return wrapper

def simple_func():
    print('this is a simple function')
    
decorating_simple_func = my_decorator(simple_func)
decorating_simple_func()

my decorator works...
this is a simple function


In [2]:
# 更简单、优雅的语法

@my_decorator
def another_func():
    print('this is another function')
    
another_func()

my decorator works...
this is another function


**这里的 @ 就是Python中装饰器的语法糖，相当于上面的原始写法。**

## 1.2 带参数的装饰器

In [3]:
# 传递函数参数

def my_decorator(func):
    def wrapper(*args, **kwargs):
        print('my decorator works, args: {}, kwargs: {}'.format(args, kwargs))
        func(*args, **kwargs)
    return wrapper

@my_decorator
def greet(message):
    print('hello {}'.format(message))
    
greet('walker')

my decorator works, args: ('walker',), kwargs: {}
hello walker


In [4]:
# 给装饰器自定义参数

def repeat(count):
    def my_decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(count):
                print('repeating decorator iterate at {} times'.format(i+1))
                func(*args, **kwargs)
        return wrapper
    return my_decorator

@repeat(3)
def say_important(message):
    print('{} is important!'.format(message))
    
say_important('You are fired')

repeating decorator iterate at 1 times
You are fired is important!
repeating decorator iterate at 2 times
You are fired is important!
repeating decorator iterate at 3 times
You are fired is important!


使用装饰器之后，原函数的元信息会被修改，这时候需要用Python内置的装饰器@functools.wraps修复，以保留原函数的元信息。

In [5]:
help(say_important)

import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('decorator works...')
        func(*args, **kwargs)
    return wrapper

@my_decorator
def greet(message):
    print(message)
    
help(greet)
greet("Hello, Walker")

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)

Help on function greet in module __main__:

greet(message)

decorator works...
Hello, Walker


## 1.3 类装饰器

类作为装饰器，主要依赖于它的`__call__`函数，也就是说要将装饰器的实现逻辑放在内置函数`__call__`里面。

In [6]:
class Counter:
    def __init__(self, func):
        self.func = func
        self.num_calls = 0
        
    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print('{} has been called {} times.'.format(self.func.__name__, self.num_calls))
        return self.func(*args, **kwargs)
        
@Counter
def say_hi():
    print('Hi, Walker')
    
say_hi()
say_hi()
say_hi()

say_hi has been called 1 times.
Hi, Walker
say_hi has been called 2 times.
Hi, Walker
say_hi has been called 3 times.
Hi, Walker


## 1.4 嵌套装饰器

可以按照顺序嵌套多个装饰器，比如：
```python
@decorator1
@decorator2
@decorator3
def some_func():
    pass
```

其执行顺序为：`decorator1(decorator2(decorator3(func)))`

In [7]:
def decorator1(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('decorator 1 works...')
        func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('decorator 2 works...')
        func(*args, **kwargs)
    return wrapper

def decorator3(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('decorator 3 works...')
        func(*args, **kwargs)
    return wrapper

@decorator1
@decorator2
@decorator3
def some_func():
    pass

some_func()

decorator 1 works...
decorator 2 works...
decorator 3 works...


## 1.5 使用场景

在Python中，装饰器的使用场景有：
1. 身份认证
2. 日志记录
3. 输入检查
4. 设置缓存
5. 等等

In [10]:
# 身份认证示例

def authenticate(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        request = args[0]
        if check_user_logged_in(request):
            print('user has logged in.')
            return func(*args, **kwargs)
        else:
            raise Exception('You have not logged in.')
    return wrapper

def check_user_logged_in(request):
    return request is not None

@authenticate
def commit_comment(request, user):
    pass

commit_comment('request', 'walker')

user has logged in.


In [14]:
# 日志记录

import time

def log_execution_time(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        end = time.perf_counter()
        print('{} took {} ms'.format(func.__name__, (end - start) * 1000))
        return res
    return wrapper
   
@functools.lru_cache()
@log_execution_time
def calculate_prime():
    pass

calculate_prime()

calculate_prime took 0.0023679895093664527 ms


# 2 生成器

生成器是Python中一个比较高级且重要的特性，它可以根据一组规则，一个接一个地生成无限个元素。

相对于列表或者元组，它最大的特点在于可以节约大量的内存空间，提高程序性能。因为在生成器中，元素是一个接一个地生成的，并不需要事先将所有元素创建出来。这在某些场景中是很有吸引力的，比如大数据的分块加载。

比如下面这个例子中，生成器版本只需要最大几十MB内存，而迭代器版本的需要接近4GB内存。

In [6]:
import os, psutil

def show_memory_info(hint):
    pid = os.getpid()
    p = psutil.Process(pid)
    
    info = p.memory_full_info()
    memory = info.uss / 1024. / 1024
    print('{} memory used: {} MB'.format(hint, memory))
def test_iterator():
    show_memory_info('initing iterator')
    list_1 = [i for i in range(100000000)]
    show_memory_info('after iterator initiated')
    print(sum(list_1))
    show_memory_info('after sum called')

def test_generator():
    show_memory_info('initing generator')
    list_2 = (i for i in range(100000000))
    show_memory_info('after generator initiated')
    print(sum(list_2))
    show_memory_info('after sum called')

test_iterator()
test_generator()

initing iterator memory used: 40.1875 MB
after iterator initiated memory used: 3674.34375 MB
4999999950000000
after sum called memory used: 3674.34375 MB
initing generator memory used: 38.21875 MB
after generator initiated memory used: 38.21875 MB
4999999950000000
after sum called memory used: 38.21875 MB


用列表生成式可以很方便的生成一个新列表，当把方括号`[]`换成圆括号`()`时，就会得到一个生成器。另外，使用关键字`yield`也可以显式创建生成器。

In [10]:
gen1 = (i ** 2 for i in range(10))

def my_generator(n):
    i = 0
    while i < n:
        yield i ** 2
        i += 1
        
gen2 = my_generator(10)

print(f"generator 1 is {gen1}, list to {list(gen1)}")
print(f"generator 2 is {gen2}, list to {list(gen2)}")

gen1 == gen2


generator 1 is <generator object <genexpr> at 0x104682a50>, list to [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
generator 2 is <generator object my_generator at 0x104682c80>, list to [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


False

和迭代器一样，生成器也是通过`next(i)`函数来遍历，当完成之后，会抛出`StopIteration`异常。

In [13]:
gen3 = my_generator(3)

print(next(gen3))
print(next(gen3))
print(next(gen3))
print(next(gen3))

0
1
4


StopIteration: 